In [1]:
import pandas as pd
import numpy as np

### Load static annual dataset and collapse MultiIndex

In [2]:
static_annual_data = pd.read_excel("../../data/4. data_processed/static_annual_data/combined_static_annual_data.xlsx", header=[0,1,2,3,4], skiprows=0)
static_annual_data.drop(index=0, inplace=True)
static_annual_data.drop(columns=static_annual_data.columns[0], inplace=True)
static_annual_data.head()

County Code            County Name   Voivodeship Code  \
  Unnamed: 0_level_1     Unnamed: 1_level_1 Unnamed: 0_level_1   
  Unnamed: 0_level_2     Unnamed: 1_level_2 Unnamed: 0_level_2   
  Unnamed: 0_level_3     Unnamed: 1_level_3 Unnamed: 0_level_3   
  Unnamed: 0_level_4     Unnamed: 1_level_4 Unnamed: 0_level_4   
1           201000.0   powiat bolesławiecki           200000.0   
2           202000.0  powiat dzierżoniowski           200000.0   
3           203000.0       powiat głogowski           200000.0   
4           204000.0        powiat górowski           200000.0   
5           205000.0        powiat jaworski           200000.0   

         Voivodeship [animal stock] total                           \
  Unnamed: 1_level_1   Unnamed: 0_level_1                            
  Unnamed: 1_level_2   Unnamed: 0_level_2                            
  Unnamed: 1_level_3         2017_by_area 2017_by_pop 2018_by_area   
  Unnamed: 1_level_4               [head]      [head]       [head]   
1       dolnośląskie              13092.0      6210.0      14222.0   
2       dolnośląskie               4809.0      7042.0       5224.0   
3       dolnośląskie               4448.0      6207.0       4832.0   
4       dolnośląskie               7409.0      2445.0       8049.0   
5       dolnośląskie               5843.0      3535.0       6348.0   

                                        ...  \
                                        ...   
                                        ...   
  2018_by_pop 2019_by_area 2019_by_pop  ...   
       [head]       [head]      [head]  ...   
1      6746.0      13746.0      6528.0  ...   
2      7613.0       5050.0      7333.0  ...   
3      6709.0       4670.0      6460.0  ...   
4      2656.0       7780.0      2517.0  ...   
5      3796.0       6135.0      3631.0  ...   

  [crop production] permanent pastures                                       \
                            area in ha                                        
                                 total                                        
                          2017_by_area 2017_by_pop 2018_by_area 2018_by_pop   
                                  [ha]        [ha]         [ha]        [ha]   
1                               1264.0       599.0       1267.0       601.0   
2                                464.0       680.0        465.0       678.0   
3                                429.0       599.0        430.0       598.0   
4                                715.0       236.0        717.0       237.0   
5                                564.0       341.0        566.0       338.0   

                                                                              
                                                                              
                                                                              
  2019_by_area 2019_by_pop 2020_by_area 2020_by_pop 2021_by_area 2021_by_pop  
          [ha]        [ha]         [ha]        [ha]         [ha]        [ha]  
1       1222.0       580.0       1297.0       604.0       2094.0       980.0  
2        449.0       652.0        476.0       669.0        769.0      1069.0  
3        415.0       574.0        441.0       592.0        711.0       955.0  
4        692.0       224.0        734.0       226.0       1185.0       367.0  
5        546.0       323.0        579.0       333.0        934.0       534.0  

[5 rows x 999 columns]

In [3]:
#remove redundant (duplicated) columns
static_annual_data.drop(columns=[('[total vehicles] road tractors','Unnamed: 0_level_1','Unnamed: 0_level_2','2017','[pcs].1'),
 ('[total vehicles] road tractors','Unnamed: 0_level_1','Unnamed: 0_level_2','2018','[pcs].1'),
 ('[total vehicles] road tractors','Unnamed: 0_level_1','Unnamed: 0_level_2','2019','[pcs].1'),
 ('[total vehicles] road tractors','Unnamed: 0_level_1','Unnamed: 0_level_2','2020','[pcs].1'),
 ('[total vehicles] road tractors','Unnamed: 0_level_1','Unnamed: 0_level_2','2021','[pcs].1')], inplace=True)

In [4]:
def clean_column_name(colname):
    #assumes a list of tuples
    templist = []
    for level in colname:
        if not level.startswith("Unnamed"):
            templist.append(level)
    if templist[0].startswith("["):
        return ", ".join(templist[:-1])
    return ", ".join(templist)

def collapse_columns(df):
    df = df.copy()
    newcols = df.columns.to_list()
    newcols = pd.Series( list(map(clean_column_name, newcols)) )
    df.columns = newcols
    return df


In [5]:
static_annual_data = collapse_columns(static_annual_data)

### Create dataframes for separate years

In [6]:
years = ['2017','2018','2019','2020','2021']
cols = {}
for y in years:
    cols[y] = []
    for c in static_annual_data.columns:
        if c in ['County Name', 'Voivodeship']:
            cols[y].append(c)
        elif c.find(y) != -1:
            if c.endswith(y):
                cols[y].append(c)
            elif c.startswith('[production of electricity]') or \
                c.startswith('[air pollution reduction systems]') or \
                c.startswith('[plants of significant nuisance]'):
                if c.endswith('_by_pop'): cols[y].append(c)
            else:
                if c not in cols[y]:
                    if c.endswith('_by_area'): cols[y].append(c)

In [7]:
static_years = {}
for y in years:
    idf = static_annual_data[cols[y]].copy()
    idf['County Name'] = idf['County Name'].apply(lambda x: x.lower()) #lower case for powiat
    idf.index = idf['County Name'] + ', ' + idf['Voivodeship'] #create powiat_voivod to match with AQ_Weather dataset
    idf.drop(columns=['County Name', 'Voivodeship'], inplace=True)
    colnames = [] 
    for c in idf.columns: #remove the year from column name
        ind = c[::-1].find(',')
        colnames.append(c[:-(ind+1)])
    idf.columns = colnames
    idf.drop(columns=['[population density] population density of the built-up and urbanized area'], inplace=True) #this column has only missing data, remove it
    static_years[y] = idf

In [8]:
static_years['2017'].head()

,[animal stock] total,[area by land] total in ha,[area by land] utilised agricultural area,[area by land] forests,[area by land] residential areas,[area by land] industrial lands,[area by land] recreational and rest areas,[area by land] lands under waters,[area by land] wasteland,[area by land] other area,...,"[vehicles by type and fuel] road tractors, petrol","[vehicles by type and fuel] road tractors, diesel oil","[vehicles by type and fuel] road tractors, Liquefied Petroleum Gas","[vehicles by type and fuel] road tractors, others","[crop production] cereals, grand total, area in ha, total","[crop production] potatoes, area in ha, total","[crop production] sugar beets, area in ha, total","[crop production] rape and turnip rape, area in ha, total","[crop production] permanent meadows, area in ha, total","[crop production] permanent pastures, area in ha, total"
"powiat bolesławiecki, dolnośląskie",13092.0,130398.0,3.896231,2.006951,0.071910,0.045761,0.026149,0.058836,0.039224,0.385701,...,2.0,609.0,0.0,6.0,34712.0,1064.0,1357.0,7940.0,8722.0,1264.0
"powiat dzierżoniowski, dolnośląskie",4809.0,47900.0,1.431234,0.737230,0.026415,0.016810,0.009606,0.021613,0.014408,0.141683,...,1.0,341.0,0.0,39.0,12751.0,391.0,499.0,2917.0,3204.0,464.0
"powiat głogowski, dolnośląskie",4448.0,44300.0,1.323656,0.681816,0.024430,0.015546,0.008884,0.019988,0.013325,0.131033,...,1.0,377.0,0.0,27.0,11793.0,361.0,461.0,2697.0,2963.0,429.0
"powiat górowski, dolnośląskie",7409.0,73799.0,2.205081,1.135839,0.040698,0.025899,0.014799,0.033298,0.022199,0.218288,...,4.0,120.0,0.0,6.0,19645.0,602.0,768.0,4494.0,4936.0,715.0
"powiat jaworski, dolnośląskie",5843.0,58198.0,1.738949,0.895734,0.032095,0.020424,0.011671,0.026259,0.017506,0.172144,...,2.0,579.0,0.0,16.0,15492.0,475.0,606.0,3544.0,3893.0,564.0


### Load AQ_Weather dataset and merge with static annual data

In [10]:
aq_weather = pd.read_csv("../../data/6. data_merged/AQ_Weather_Merged_Rev1.csv")
aq_weather.head()

,powiat_voivod,DATE,LAT,LON,NO2_24H_AVG_POLLUTION,O3_24H_AVG_POLLUTION,PM10_24H_AVG_POLLUTION,PM25_24H_AVG_POLLUTION,CC,FG,HU,RR,SD,TG,voivodship
0,"powiat aleksandrowski, kujawsko-pomorskie",2017-01-01,52.888422,18.780908,NaN,32.22,24.968064,NaN,4.746750,0.0,88.4,1.0,0.0,1.8,kujawsko-pomorskie
1,"powiat aleksandrowski, kujawsko-pomorskie",2017-01-02,52.888422,18.780908,NaN,34.62,17.943745,NaN,6.167555,36.8,88.4,1.0,0.0,0.5,kujawsko-pomorskie
2,"powiat aleksandrowski, kujawsko-pomorskie",2017-01-03,52.888422,18.780908,NaN,42.00,14.477950,NaN,5.198131,0.0,88.4,63.0,0.0,0.3,kujawsko-pomorskie
3,"powiat aleksandrowski, kujawsko-pomorskie",2017-01-04,52.888422,18.780908,NaN,57.46,8.418471,NaN,8.432231,64.2,88.4,10.0,0.0,1.2,kujawsko-pomorskie
4,"powiat aleksandrowski, kujawsko-pomorskie",2017-01-05,52.888422,18.780908,NaN,64.62,13.189740,NaN,4.992977,0.0,82.4,2.0,1.0,-3.9,kujawsko-pomorskie


In [11]:
newcolumns = pd.Index(list(aq_weather.columns) + list(static_years['2017'].columns))

In [12]:
from tqdm import tqdm
rows = []
cache = None
for i, r in tqdm(aq_weather.iterrows()):
    pow_voi = r[0]
    year = r[1][:4]
    assert year in years
    if (pow_voi, year) != cache:
        cache = (pow_voi, year)
        static_data_pow_year = static_years[year].loc[pow_voi,:]
    newrow = np.hstack([np.array(r),  np.array(static_data_pow_year)])
    rows.append(newrow)

361548it [00:22, 15903.04it/s]


In [13]:
aq_weather_static = pd.DataFrame(np.array(rows))
aq_weather_static.columns = newcolumns
rows = [] #delete from memory

In [14]:
aq_weather_static.head()

,powiat_voivod,DATE,LAT,LON,NO2_24H_AVG_POLLUTION,O3_24H_AVG_POLLUTION,PM10_24H_AVG_POLLUTION,PM25_24H_AVG_POLLUTION,CC,FG,...,"[vehicles by type and fuel] road tractors, petrol","[vehicles by type and fuel] road tractors, diesel oil","[vehicles by type and fuel] road tractors, Liquefied Petroleum Gas","[vehicles by type and fuel] road tractors, others","[crop production] cereals, grand total, area in ha, total","[crop production] potatoes, area in ha, total","[crop production] sugar beets, area in ha, total","[crop production] rape and turnip rape, area in ha, total","[crop production] permanent meadows, area in ha, total","[crop production] permanent pastures, area in ha, total"
0,"powiat aleksandrowski, kujawsko-pomorskie",2017-01-01,52.888422,18.780908,NaN,32.22,24.968064,NaN,4.74675,0.0,...,2.0,303.0,2.0,21.0,16319.0,636.0,1184.0,2616.0,2320.0,406.0
1,"powiat aleksandrowski, kujawsko-pomorskie",2017-01-02,52.888422,18.780908,NaN,34.62,17.943745,NaN,6.167555,36.8,...,2.0,303.0,2.0,21.0,16319.0,636.0,1184.0,2616.0,2320.0,406.0
2,"powiat aleksandrowski, kujawsko-pomorskie",2017-01-03,52.888422,18.780908,NaN,42.0,14.47795,NaN,5.198131,0.0,...,2.0,303.0,2.0,21.0,16319.0,636.0,1184.0,2616.0,2320.0,406.0
3,"powiat aleksandrowski, kujawsko-pomorskie",2017-01-04,52.888422,18.780908,NaN,57.46,8.418471,NaN,8.432231,64.2,...,2.0,303.0,2.0,21.0,16319.0,636.0,1184.0,2616.0,2320.0,406.0
4,"powiat aleksandrowski, kujawsko-pomorskie",2017-01-05,52.888422,18.780908,NaN,64.62,13.18974,NaN,4.992977,0.0,...,2.0,303.0,2.0,21.0,16319.0,636.0,1184.0,2616.0,2320.0,406.0


In [15]:
#check columns with null values
for col in aq_weather_static.columns:
    if aq_weather_static[col].isnull().any():
        print(col, " contains null values")

NO2_24H_AVG_POLLUTION  contains null values
O3_24H_AVG_POLLUTION  contains null values
PM10_24H_AVG_POLLUTION  contains null values
PM25_24H_AVG_POLLUTION  contains null values


In [16]:
aq_weather_static.to_csv("../../data/6. data_merged/aq_weather_static_merged.csv", index=False)